# Wstępna obróbka datasetu

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Attention, Permute, Multiply, Lambda
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


## 1a. Wczytaj dataset z filmików lekotrów

### Wczytanie datasetu

In [ ]:
df = pd.read_csv('videos/videos_dataset.csv')

In [ ]:
df.head()

,sample_id,frame,user_id,label,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,...,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z,landmark_21_x,landmark_21_y,landmark_21_z
0,1,0,a2,A,0.345417,0.648431,-1.125724e-07,0.363545,0.620765,-0.010274,...,-0.003865,0.321722,0.551512,-0.008457,0.326903,0.565844,-0.005724,0.329682,0.577548,-0.001789
1,1,1,a2,A,0.366266,0.597102,-9.622672e-08,0.391580,0.578109,-0.012907,...,0.000685,0.356418,0.497935,-0.004451,0.361665,0.515749,-0.001988,0.363353,0.523833,0.002337
2,1,2,a2,A,0.400966,0.572690,-1.089629e-07,0.424555,0.552354,-0.004017,...,-0.004296,0.383425,0.481909,-0.009304,0.389207,0.502081,-0.005430,0.391897,0.513775,-0.000924
3,1,3,a2,A,0.418257,0.573744,-1.190043e-07,0.441344,0.563031,-0.007786,...,-0.002903,0.402536,0.489501,-0.009669,0.406139,0.510639,-0.006567,0.409394,0.523706,-0.001918
4,1,4,a2,A,0.431913,0.582072,-1.241054e-07,0.453692,0.577044,-0.009167,...,-0.004793,0.417201,0.501486,-0.012369,0.419653,0.524298,-0.008715,0.422120,0.535854,-0.003439


In [ ]:
df.drop('user_id', axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13080 entries, 0 to 13079
Data columns (total 66 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sample_id      13080 non-null  int64  
 1   frame          13080 non-null  int64  
 2   label          13080 non-null  object 
 3   landmark_1_x   13080 non-null  float64
 4   landmark_1_y   13080 non-null  float64
 5   landmark_1_z   13080 non-null  float64
 6   landmark_2_x   13080 non-null  float64
 7   landmark_2_y   13080 non-null  float64
 8   landmark_2_z   13080 non-null  float64
 9   landmark_3_x   13080 non-null  float64
 10  landmark_3_y   13080 non-null  float64
 11  landmark_3_z   13080 non-null  float64
 12  landmark_4_x   13080 non-null  float64
 13  landmark_4_y   13080 non-null  float64
 14  landmark_4_z   13080 non-null  float64
 15  landmark_5_x   13080 non-null  float64
 16  landmark_5_y   13080 non-null  float64
 17  landmark_5_z   13080 non-null  float64
 18  landma

In [ ]:
# set label as category type
df['label'] = df['label'].astype('category')

## 1b. Wczytaj dataset moj dataset

In [ ]:
df2 = pd.read_csv('own/own_dataset.csv')

In [ ]:
df2.head()

,user_id,label,sample_id,frame,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,...,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z,landmark_21_x,landmark_21_y,landmark_21_z
0,1,Z,1,0,0.435695,0.576969,4.036861e-08,0.480515,0.548350,-0.026277,...,-0.048237,0.413569,0.455929,-0.074037,0.423309,0.496448,-0.069420,0.417086,0.505802,-0.059601
1,3,Z,1,0,0.578351,0.609560,5.265319e-08,0.624348,0.585140,-0.014683,...,-0.061790,0.585376,0.526921,-0.080099,0.590219,0.553543,-0.076534,0.579452,0.550328,-0.069926
2,4,Z,1,0,0.507152,0.550208,6.161761e-08,0.538633,0.521261,-0.014309,...,-0.027720,0.487526,0.448841,-0.044816,0.503914,0.473505,-0.041604,0.512892,0.491349,-0.033786
3,1,Z,1,1,0.437542,0.579177,4.156175e-08,0.482692,0.549850,-0.026735,...,-0.047561,0.414696,0.457763,-0.073083,0.424467,0.497505,-0.068846,0.418851,0.507512,-0.059264
4,3,Z,1,1,0.578379,0.609218,5.853659e-08,0.624059,0.584968,-0.014740,...,-0.061531,0.585250,0.526183,-0.079531,0.589327,0.552980,-0.075569,0.577682,0.549286,-0.068727


In [ ]:
df2 = df2.drop('user_id', axis=1)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32463 entries, 0 to 32462
Data columns (total 66 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   label          32463 non-null  object 
 1   sample_id      32463 non-null  int64  
 2   frame          32463 non-null  int64  
 3   landmark_1_x   32463 non-null  float64
 4   landmark_1_y   32463 non-null  float64
 5   landmark_1_z   32463 non-null  float64
 6   landmark_2_x   32463 non-null  float64
 7   landmark_2_y   32463 non-null  float64
 8   landmark_2_z   32463 non-null  float64
 9   landmark_3_x   32463 non-null  float64
 10  landmark_3_y   32463 non-null  float64
 11  landmark_3_z   32463 non-null  float64
 12  landmark_4_x   32463 non-null  float64
 13  landmark_4_y   32463 non-null  float64
 14  landmark_4_z   32463 non-null  float64
 15  landmark_5_x   32463 non-null  float64
 16  landmark_5_y   32463 non-null  float64
 17  landmark_5_z   32463 non-null  float64
 18  landma

In [ ]:
df2['label'] = df2['label'].astype('category')

## 1c. Połączenie datasetów

In [ ]:
# 1. Liczba unikalnych próbek w df
max_sample_id_df = df['sample_id'].max()

# 2. Dodaj przesunięcie do df2
df2['sample_id'] += max_sample_id_df

# 3. Połączenie obu zbiorów
merged_df = pd.concat([df, df2], ignore_index=True)

# (opcjonalnie) Sortuj po sample_id i frame, jeśli potrzebne
merged_df.sort_values(by=['sample_id', 'frame'], inplace=True)

# Przykład zapisu
merged_df.to_csv('full_datasets.csv', index=False)

In [ ]:
len(df) + len(df2)

45543

In [ ]:
len(merged_df)

45543